In [1]:
#autoreload
%load_ext autoreload
%autoreload 2

import torch
from torchmetrics import AUROC
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import os

import sys
sys.path.append("../../src_clasificacion_vistas/common")

import pycimg

from pathlib import Path
import cv2
from tqdm import tqdm
import numpy as np
import gc
np.set_printoptions(precision=3, suppress=True)

In [2]:
def crearMosaico(imagenes):
    """
    Crea un mosaico a partir de una lista de imágenes.
    """

    #Convertir las imágenes de tensores c x h x w float32 entre 0 y 1 a uint8 entre 0 y 255 h x w x c
    for i in range(len(imagenes)):
        imagenes[i] = (imagenes[i].permute(1, 2, 0).numpy() * 255).astype(np.uint8)
    
    # Obtener el tamaño de la imagen
    alto, ancho = imagenes[0].shape[:2]
    
    # Redimensionar todas las imágenes al tamaño de la primera
    # imagenes[0] = cv2.cvtColor(imagenes[0], cv2.COLOR_BGR2RGB)
    for i in range(1, len(imagenes)):
        imagenes[i] = cv2.resize(imagenes[i], (ancho, alto))
    #     imagenes[i] = cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2RGB)

    # Calcular el número de filas y columnas
    num_imagenes = len(imagenes)
    num_columnas = int(np.ceil(np.sqrt(num_imagenes)))
    num_filas = int(np.ceil(num_imagenes / num_columnas))
    #print(f"num_imagenes: {num_imagenes}, num_columnas: {num_columnas}, num_filas: {num_filas}")

    # Crear una imagen en blanco para el mosaico
    mosaico = np.zeros((alto * num_filas, ancho * num_columnas, 3), dtype=np.uint8)

    # Colocar las imágenes en el mosaico
    for i, img in enumerate(imagenes):
        fila = i // num_columnas
        columna = i % num_columnas
        mosaico[fila * alto:(fila + 1) * alto, columna * ancho:(columna + 1) * ancho] = img

    return mosaico

In [12]:
current_file_dir = os.getcwd()
train_results_json="../out_evaluate_patch_repasoD/train_scores.json"
val_results_json="../out_evaluate_patch_repasoD/val_scores.json"
train_results_json=os.path.join(current_file_dir,train_results_json)    
val_results_json=os.path.join(current_file_dir,val_results_json)    
out_dir="/home/aalbiol/Desktop/repasoE"

Path(out_dir).mkdir(parents=True, exist_ok=True)



def create_errores(train_data,clases):

    matrices_confusion={}
    for clase in clases:
        matrices_confusion[clase]=np.zeros((2,2))
    k=0
    for d in tqdm(train_data):
        k+=1
        filenamejson=d['filename'].replace(".npz",".json")
        scores=d['scores']
        ground_truth=d['ground_truth']
        kk=os.path.join("..",filenamejson)
        npzfilename=kk.replace(".json",".npz")
        imagenes=pycimg.npzread_torch(npzfilename,kk,channel_list=['R','G','B'])
        mosaico=crearMosaico(imagenes)
# Analizar todas las etiquetas
        gts=[]
        preds=[]
        defectoss=[]
        for m, defecto in enumerate(clases):
            if defecto in scores:
                gt=ground_truth[defecto]
                pred=scores[defecto]
                gts.append(gt)
                preds.append(pred)
                defectoss.append(defecto)
        #Generar imágenes de los errores
        for m, defecto in enumerate(clases):
            if not defecto in scores:
                continue
            gt=ground_truth[defecto]
            pred=scores[defecto]

            directorio_clase=os.path.join(out_dir,defecto)
            directorio=None
            if pred<=0.5:
                pred_matriz=0
            else:
                pred_matriz=1
            matrices_confusion[defecto][int(gt)][pred_matriz]+=1
                
            if gt>0.5 and pred <=0.5:
                if np.sum(np.array(gts))>1:
                    directorio=os.path.join(directorio_clase,"falsos_negativos_con_otro_defecto")                        
                else:
                    directorio=os.path.join(directorio_clase,"falsos_negativos_sin_otro_defecto")                    
                
            elif gt<0.5 and pred >0.5:
                if max(gts)>0.5:
                    directorio=os.path.join(directorio_clase,"falsos_positivos_con_otro_defecto")    
                else:
                    directorio=os.path.join(directorio_clase,"falsos_positivos_sin_ningun_defecto")
                    
            bn=os.path.basename(filenamejson)
            bn=bn.replace(".json","")        
            #fig, axes=plt.subplots(1,2,gridspec_kw={'width_ratios': [1, 2]})
            fig, axes=plt.subplots(1,2,figsize=(20,10))
            axes[0].bar(defectoss,gts)
            axes[0].bar(defectoss,preds)
    
            axes[0].tick_params(axis='x',rotation=90)
            axes[0].set_ylim([0,1])
            axes[0].set_yticks(np.arange(0, 1.1, 0.1))
            axes[0].grid()
    


            axes[1].imshow(mosaico)
            axes[1].axis('off')
            axes[1].set_title(f'{bn} - {defecto} :{int(1000*pred)}')                    
            if directorio is not None:
                Path(directorio).mkdir(parents=True, exist_ok=True)
                
                bn=f'{bn}_{int(1000*pred)}.jpg'
                nombre_fichero=os.path.join(directorio,bn)
                #print(f"Guardando {nombre_fichero}")
                fig.savefig(nombre_fichero, bbox_inches='tight', dpi=300)
            plt.close('all')
            del fig, axes

                
                
    for defecto in clases:
        print(f">>>>>>>>>>>>>>>> Defecto: {defecto}")
        matriz=matrices_confusion[defecto]
        print(matriz)
        # print(f"Verdaderos positivos: {matriz[1][1]}")
        # print(f"Falsos negativos: {matriz[1][0]}")
        # print(f"Falsos positivos: {matriz[0][1]}")
        # print(f"Verdaderos negativos: {matriz[0][0]}")
        # print(f"Total: {np.sum(matriz)}")
        # print(f"Precisión: {matriz[1][1]/(matriz[1][1]+matriz[0][1])}")
        # print(f"Recall: {matriz[1][1]/(matriz[1][1]+matriz[1][0])}")
        # print(f"F1: {2*matriz[1][1]/(2*matriz[1][1]+matriz[0][1]+matriz[1][0])}")

                
        
                    






In [4]:
## Leer los archivos

with open(train_results_json, "r") as f:
    train_dict = json.load(f)

with open(val_results_json, "r") as f:
    val_dict = json.load(f)    

train_data=train_dict['train_results']
val_data = val_dict['val_results']
clases=list(train_dict['train_results'][0]['ground_truth'].keys())

all_data=train_data+val_data

print(len(train_data))
print(len(val_data))
print(len(all_data))

1283
343
1626


In [6]:
create_errores(all_data[:500],clases)

100%|██████████| 500/500 [04:24<00:00,  1.89it/s]

>>>>>>>>>>>>>>>> Defecto: cracks
[[477.   7.]
 [  3.  13.]]
>>>>>>>>>>>>>>>> Defecto: damage_by_pliers
[[448.  19.]
 [ 12.  21.]]
>>>>>>>>>>>>>>>> Defecto: deformed_peduncle
[[377.  17.]
 [  4. 102.]]
>>>>>>>>>>>>>>>> Defecto: green
[[408.  31.]
 [  1.  60.]]
>>>>>>>>>>>>>>>> Defecto: hollow
[[459.  11.]
 [  2.  28.]]
>>>>>>>>>>>>>>>> Defecto: insect
[[482.  13.]
 [  0.   5.]]
>>>>>>>>>>>>>>>> Defecto: light_scars
[[415.  17.]
 [ 12.  56.]]
>>>>>>>>>>>>>>>> Defecto: mechanical_damage
[[492.   6.]
 [  1.   1.]]
>>>>>>>>>>>>>>>> Defecto: oil_spots
[[451.   8.]
 [  6.  35.]]
>>>>>>>>>>>>>>>> Defecto: plu_stickers
[[500.   0.]
 [  0.   0.]]
>>>>>>>>>>>>>>>> Defecto: reddish
[[499.   0.]
 [  1.   0.]]
>>>>>>>>>>>>>>>> Defecto: rotten
[[430.  14.]
 [  7.  49.]]
>>>>>>>>>>>>>>>> Defecto: scars
[[449.  17.]
 [  3.  31.]]
>>>>>>>>>>>>>>>> Defecto: skin_breakdown
[[412.  35.]
 [  9.  44.]]
>>>>>>>>>>>>>>>> Defecto: surface
[[387.  28.]
 [  5.  80.]]


In [7]:
create_errores(all_data[500:800],clases)

100%|██████████| 300/300 [02:43<00:00,  1.83it/s]

>>>>>>>>>>>>>>>> Defecto: cracks
[[286.   3.]
 [  2.   9.]]
>>>>>>>>>>>>>>>> Defecto: damage_by_pliers
[[285.   4.]
 [  4.   7.]]
>>>>>>>>>>>>>>>> Defecto: deformed_peduncle
[[298.   1.]
 [  0.   1.]]
>>>>>>>>>>>>>>>> Defecto: green
[[221.  45.]
 [  0.  34.]]
>>>>>>>>>>>>>>>> Defecto: hollow
[[289.   5.]
 [  1.   5.]]
>>>>>>>>>>>>>>>> Defecto: insect
[[287.   8.]
 [  0.   5.]]
>>>>>>>>>>>>>>>> Defecto: light_scars
[[266.   9.]
 [  1.  24.]]
>>>>>>>>>>>>>>>> Defecto: mechanical_damage
[[297.   2.]
 [  0.   1.]]
>>>>>>>>>>>>>>>> Defecto: oil_spots
[[280.  10.]
 [  5.   5.]]
>>>>>>>>>>>>>>>> Defecto: plu_stickers
[[299.   1.]
 [  0.   0.]]
>>>>>>>>>>>>>>>> Defecto: reddish
[[298.   0.]
 [  2.   0.]]
>>>>>>>>>>>>>>>> Defecto: rotten
[[263.   5.]
 [  2.  30.]]
>>>>>>>>>>>>>>>> Defecto: scars
[[242.  19.]
 [  4.  35.]]
>>>>>>>>>>>>>>>> Defecto: skin_breakdown
[[259.  17.]
 [  2.  22.]]
>>>>>>>>>>>>>>>> Defecto: surface
[[184.  21.]
 [  3.  92.]]


In [10]:
gc.collect()
create_errores(all_data[800:1100],clases)

100%|██████████| 300/300 [03:01<00:00,  1.66it/s]

>>>>>>>>>>>>>>>> Defecto: cracks
[[282.   4.]
 [  1.  13.]]
>>>>>>>>>>>>>>>> Defecto: damage_by_pliers
[[275.  10.]
 [  3.  12.]]
>>>>>>>>>>>>>>>> Defecto: deformed_peduncle
[[295.   3.]
 [  1.   1.]]
>>>>>>>>>>>>>>>> Defecto: green
[[263.  24.]
 [  0.  13.]]
>>>>>>>>>>>>>>>> Defecto: hollow
[[295.   2.]
 [  1.   2.]]
>>>>>>>>>>>>>>>> Defecto: insect
[[282.  17.]
 [  0.   1.]]
>>>>>>>>>>>>>>>> Defecto: light_scars
[[244.   6.]
 [ 17.  33.]]
>>>>>>>>>>>>>>>> Defecto: mechanical_damage
[[275.  14.]
 [  3.   8.]]
>>>>>>>>>>>>>>>> Defecto: oil_spots
[[269.   9.]
 [  7.  15.]]
>>>>>>>>>>>>>>>> Defecto: plu_stickers
[[248.   5.]
 [  1.  46.]]
>>>>>>>>>>>>>>>> Defecto: reddish
[[290.   5.]
 [  3.   2.]]
>>>>>>>>>>>>>>>> Defecto: rotten
[[263.  11.]
 [  2.  24.]]
>>>>>>>>>>>>>>>> Defecto: scars
[[198.  56.]
 [  5.  41.]]
>>>>>>>>>>>>>>>> Defecto: skin_breakdown
[[229.  19.]
 [  8.  44.]]
>>>>>>>>>>>>>>>> Defecto: surface
[[211.  15.]
 [  7.  67.]]


In [ ]:
gc.collect()
create_errores(all_data[1100:1350],clases)

100%|██████████| 250/250 [02:30<00:00,  1.66it/s]

>>>>>>>>>>>>>>>> Defecto: cracks
[[240.   4.]
 [  1.   5.]]
>>>>>>>>>>>>>>>> Defecto: damage_by_pliers
[[226.  11.]
 [  3.  10.]]
>>>>>>>>>>>>>>>> Defecto: deformed_peduncle
[[189.  12.]
 [  4.  45.]]
>>>>>>>>>>>>>>>> Defecto: green
[[244.   1.]
 [  0.   5.]]
>>>>>>>>>>>>>>>> Defecto: hollow
[[241.   5.]
 [  0.   4.]]
>>>>>>>>>>>>>>>> Defecto: insect
[[233.  11.]
 [  0.   6.]]
>>>>>>>>>>>>>>>> Defecto: light_scars
[[163.  14.]
 [ 10.  63.]]
>>>>>>>>>>>>>>>> Defecto: mechanical_damage
[[196.  14.]
 [  4.  36.]]
>>>>>>>>>>>>>>>> Defecto: oil_spots
[[233.   5.]
 [  3.   9.]]
>>>>>>>>>>>>>>>> Defecto: plu_stickers
[[216.   1.]
 [  0.  33.]]
>>>>>>>>>>>>>>>> Defecto: reddish
[[214.   3.]
 [  7.  26.]]
>>>>>>>>>>>>>>>> Defecto: rotten
[[238.   7.]
 [  0.   5.]]
>>>>>>>>>>>>>>>> Defecto: scars
[[164.  24.]
 [  7.  55.]]
>>>>>>>>>>>>>>>> Defecto: skin_breakdown
[[194.  26.]
 [  6.  24.]]
>>>>>>>>>>>>>>>> Defecto: surface
[[198.  24.]
 [  1.  27.]]


In [13]:
gc.collect()
create_errores(all_data[1350:],clases)

100%|██████████| 276/276 [02:51<00:00,  1.61it/s]

>>>>>>>>>>>>>>>> Defecto: cracks
[[260.   4.]
 [  3.   9.]]
>>>>>>>>>>>>>>>> Defecto: damage_by_pliers
[[252.  10.]
 [  3.  11.]]
>>>>>>>>>>>>>>>> Defecto: deformed_peduncle
[[258.  10.]
 [  3.   5.]]
>>>>>>>>>>>>>>>> Defecto: green
[[224.  30.]
 [  6.  16.]]
>>>>>>>>>>>>>>>> Defecto: hollow
[[261.   3.]
 [  2.  10.]]
>>>>>>>>>>>>>>>> Defecto: insect
[[267.   6.]
 [  0.   3.]]
>>>>>>>>>>>>>>>> Defecto: light_scars
[[215.  16.]
 [  9.  36.]]
>>>>>>>>>>>>>>>> Defecto: mechanical_damage
[[248.  12.]
 [  5.  11.]]
>>>>>>>>>>>>>>>> Defecto: oil_spots
[[252.   9.]
 [  5.  10.]]
>>>>>>>>>>>>>>>> Defecto: plu_stickers
[[247.   2.]
 [  0.  27.]]
>>>>>>>>>>>>>>>> Defecto: reddish
[[264.   2.]
 [  5.   5.]]
>>>>>>>>>>>>>>>> Defecto: rotten
[[232.  18.]
 [  6.  20.]]
>>>>>>>>>>>>>>>> Defecto: scars
[[204.  36.]
 [  2.  34.]]
>>>>>>>>>>>>>>>> Defecto: skin_breakdown
[[221.  29.]
 [  4.  22.]]
>>>>>>>>>>>>>>>> Defecto: surface
[[188.  25.]
 [  2.  61.]]


In [9]:
current_file_dir = os.getcwd()
train_results_json="../out_evaluate_patch_repasoD/train_scores.json"
val_results_json="../out_evaluate_patch_repasoD/val_scores.json"
train_results_json=os.path.join(current_file_dir,train_results_json)    
val_results_json=os.path.join(current_file_dir,val_results_json)    
out_dir="/home/aalbiol/Desktop/repasoE-val"

Path(out_dir).mkdir(parents=True, exist_ok=True)

In [10]:
create_errores(val_data,clases)

100%|██████████| 343/343 [00:03<00:00, 112.27it/s]

>>>>>>>>>>>>>>>> Defecto: cracks
[[315.  15.   1.]
 [  2.   2.   8.]]
>>>>>>>>>>>>>>>> Defecto: damage_by_pliers
[[296.  28.   3.]
 [  2.   9.   5.]]
>>>>>>>>>>>>>>>> Defecto: deformed_peduncle
[[283.  23.   2.]
 [  2.   4.  29.]]
>>>>>>>>>>>>>>>> Defecto: green
[[266.  39.  11.]
 [  3.   3.  21.]]
>>>>>>>>>>>>>>>> Defecto: hollow
[[301.  26.   0.]
 [  0.   4.  12.]]
>>>>>>>>>>>>>>>> Defecto: insect
[[323.  14.   3.]
 [  0.   2.   1.]]
>>>>>>>>>>>>>>>> Defecto: light_scars
[[257.  30.   6.]
 [  5.  19.  26.]]
>>>>>>>>>>>>>>>> Defecto: mechanical_damage
[[298.  25.   4.]
 [  1.   5.  10.]]
>>>>>>>>>>>>>>>> Defecto: oil_spots
[[294.  25.   3.]
 [  3.   7.  11.]]
>>>>>>>>>>>>>>>> Defecto: plu_stickers
[[309.   7.   0.]
 [  0.   0.  27.]]
>>>>>>>>>>>>>>>> Defecto: reddish
[[324.   8.   1.]
 [  4.   3.   3.]]
>>>>>>>>>>>>>>>> Defecto: rotten
[[285.  22.   6.]
 [  3.   8.  19.]]
>>>>>>>>>>>>>>>> Defecto: scars
[[239.  43.  19.]
 [  1.  12.  29.]]
>>>>>>>>>>>>>>>> Defecto: skin_breakdown
[[25